In [1]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
#ratings_df = pd.read_csv('resource/ml-25m/ratings.csv')
disney_df = pd.read_csv('../resource/disney/disney_movies_total_gross.csv')
disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000","$920,608,730"


In [3]:
disney_df.count()

movie_title                 579
release_date                579
genre                       562
MPAA_rating                 523
total_gross                 579
inflation_adjusted_gross    579
dtype: int64

In [4]:
disney_list_df = disney_df['movie_title']

In [5]:
#movie lens database
movies_df = pd.read_csv('../resource/ml-25m/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies_df['title'] = movies_df['title'].astype(str)

In [7]:
movies_df.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [8]:
#cleanup movie title
movies_df['movie_title'] = movies_df['title'].str.split(' \(', 1).str[0]
movies_df.head()

,movieId,title,genres,movie_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II


In [9]:
movies_df.drop('title', axis=1, inplace=True)
movies_df.head()

,movieId,genres,movie_title
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Adventure|Children|Fantasy,Jumanji
2,3,Comedy|Romance,Grumpier Old Men
3,4,Comedy|Drama|Romance,Waiting to Exhale
4,5,Comedy,Father of the Bride Part II


In [10]:
merged_disney_df = pd.merge(disney_df, movies_df, on="movie_title")
merged_disney_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,movieId,genres
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",594,Animation|Children|Drama|Fantasy|Musical
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",596,Animation|Children|Fantasy|Musical
2,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",5990,Children|Comedy|Fantasy
3,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",90264,Children|Fantasy|Musical
4,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052",126693,Animation|Children


In [11]:
merged_disney_df.count()

movie_title                 596
release_date                596
genre                       582
MPAA_rating                 532
total_gross                 596
inflation_adjusted_gross    596
movieId                     596
genres                      596
dtype: int64

In [12]:
merged_disney_df.drop('genre', axis=1, inplace=True)
merged_disney_df.head()

,movie_title,release_date,MPAA_rating,total_gross,inflation_adjusted_gross,movieId,genres
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",G,"$184,925,485","$5,228,953,251",594,Animation|Children|Drama|Fantasy|Musical
1,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",596,Animation|Children|Fantasy|Musical
2,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",5990,Children|Comedy|Fantasy
3,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",90264,Children|Fantasy|Musical
4,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",126693,Animation|Children


In [13]:
merged_disney_df.to_csv (r'output/MERGED_disney_movies_total_gross.csv', index = None, header=True) 

In [14]:
#movie lens database
ratings_df = pd.read_csv('../resource/ml-25m/ratings.csv')
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [15]:
merged_disney_ratings = pd.merge(ratings_df, merged_disney_df, on="movieId")
merged_disney_ratings.head()

,userId,movieId,rating,timestamp,movie_title,release_date,MPAA_rating,total_gross,inflation_adjusted_gross,genres
0,1,6377,4.0,1147868469,Finding Nemo,"May 30, 2003",G,"$380,529,370","$518,148,559",Adventure|Animation|Children|Comedy
1,3,6377,4.0,1439472644,Finding Nemo,"May 30, 2003",G,"$380,529,370","$518,148,559",Adventure|Animation|Children|Comedy
2,4,6377,3.0,1573939584,Finding Nemo,"May 30, 2003",G,"$380,529,370","$518,148,559",Adventure|Animation|Children|Comedy
3,12,6377,4.0,1167582434,Finding Nemo,"May 30, 2003",G,"$380,529,370","$518,148,559",Adventure|Animation|Children|Comedy
4,13,6377,4.0,1238029123,Finding Nemo,"May 30, 2003",G,"$380,529,370","$518,148,559",Adventure|Animation|Children|Comedy


In [16]:
merged_disney_ratings.isnull().sum()

userId                          0
movieId                         0
rating                          0
timestamp                       0
movie_title                     0
release_date                    0
MPAA_rating                 61362
total_gross                     0
inflation_adjusted_gross        0
genres                          0
dtype: int64

In [17]:
merged_disney_ratings.count()

userId                      1624064
movieId                     1624064
rating                      1624064
timestamp                   1624064
movie_title                 1624064
release_date                1624064
MPAA_rating                 1562702
total_gross                 1624064
inflation_adjusted_gross    1624064
genres                      1624064
dtype: int64

In [18]:
merged_disney_ratings.to_csv (r'output/MERGED_disney_ratings.csv', index = None, header=True) 

## Create some test data for KNN

will not need the codes below for production

## FOR TESTING ONLY

In [19]:
## TEST ONLY RATINGS
#get random 15K sample ratings
plot_disney_ratings = merged_disney_ratings.sample(n=15000, random_state=1)

#save to csv
plot_disney_ratings.to_csv (r'output/test_plot_disney_ratings.csv', index = None, header=True) 

In [20]:
plot_disney_ratings.head()

,userId,movieId,rating,timestamp,movie_title,release_date,MPAA_rating,total_gross,inflation_adjusted_gross,genres
1360739,89623,1730,2.5,1241365261,Kundun,"Dec 25, 1997",PG-13,"$5,686,694","$10,236,836",Drama
1562144,94673,55267,4.0,1219570648,Dan in Real Life,"Oct 20, 2007",PG-13,"$47,642,963","$58,350,234",Comedy|Drama|Romance
929376,125508,1367,4.0,853764884,101 Dalmatians,"Nov 27, 1996",G,"$136,189,294","$258,728,898",Adventure|Children|Comedy
694197,15892,595,4.0,962784074,Beauty and the Beast,"Nov 13, 1991",G,"$218,951,625","$363,017,667",Animation|Children|Fantasy|Musical|Romance|IMAX
452264,464,78499,3.5,1448668642,Toy Story 3,"Jun 18, 2010",G,"$415,004,880","$443,408,255",Adventure|Animation|Children|Comedy|Fantasy|IMAX


In [21]:
disney_ratings = merged_disney_ratings[['userId', 'movieId', 'rating', 'timestamp']].copy()

## TEST ONLY RATINGS
#get random 15K sample ratings
test_disney_ratings = disney_ratings.sample(n=15000, random_state=1)

#save to csv
test_disney_ratings.to_csv (r'output/test_disney_ratings.csv', index = None, header=True) 

In [22]:
test_disney_ratings['userId'].nunique()

13196

In [23]:
test_disney_ratings.head()

,userId,movieId,rating,timestamp
1360739,89623,1730,2.5,1241365261
1562144,94673,55267,4.0,1219570648
929376,125508,1367,4.0,853764884
694197,15892,595,4.0,962784074
452264,464,78499,3.5,1448668642


## PRODUCTION DATASET STARTS HERE

In [25]:
disney_ratings = merged_disney_ratings[['userId', 'movieId', 'rating', 'timestamp']].copy()
#production ratings
disney_ratings.to_csv (r'output/disney_ratings.csv', index = None, header=True) 

In [26]:
disney_ratings['userId'].nunique()

150172

In [27]:
disney_ratings['userId'].count()

1624064

In [28]:
merged_disney_df.rename(columns={'movie_title': 'title'}, inplace = True)
merged_disney_df.head()

,title,release_date,MPAA_rating,total_gross,inflation_adjusted_gross,movieId,genres
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",G,"$184,925,485","$5,228,953,251",594,Animation|Children|Drama|Fantasy|Musical
1,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",596,Animation|Children|Fantasy|Musical
2,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",5990,Children|Comedy|Fantasy
3,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",90264,Children|Fantasy|Musical
4,Pinocchio,"Feb 9, 1940",G,"$84,300,000","$2,188,229,052",126693,Animation|Children


In [32]:
disney_movies = merged_disney_df[['movieId', 'title', 'genres']].copy()
disney_movies.to_csv (r'output/disney_movies.csv', index = None, header=True) 

In [30]:
test_disney_movies.head()

,movieId,title,genres
0,594,Snow White and the Seven Dwarfs,Animation|Children|Drama|Fantasy|Musical
1,596,Pinocchio,Animation|Children|Fantasy|Musical
2,5990,Pinocchio,Children|Comedy|Fantasy
3,90264,Pinocchio,Children|Fantasy|Musical
4,126693,Pinocchio,Animation|Children


In [31]:
test_disney_movies.count()

movieId    596
title      596
genres     596
dtype: int64

In [ ]:
#df[df.duplicated(keep=False)]